In [ ]:
import pandas as pd


In [4]:
df = pd.read_csv("data_meteo_toulouse_station/data_all_stations.csv")

In [5]:
df

,id,heure_de_paris,humidite,pression,temperature_en_degre_c,station_name
0,12,2025-11-06T14:00:00+00:00,92,98900,12.9,montaudran
1,12,2025-11-06T13:00:00+00:00,92,98900,12.7,montaudran
2,12,2025-11-06T12:00:00+00:00,93,98900,12.4,montaudran
3,12,2025-11-06T11:00:00+00:00,92,98900,12.7,montaudran
4,12,2025-11-06T10:00:00+00:00,92,99000,12.9,montaudran
...,...,...,...,...,...,...
92,12,2025-11-02T18:00:00+00:00,85,100400,13.9,montaudran
93,12,2025-11-02T17:00:00+00:00,83,100300,14.4,montaudran
94,12,2025-11-02T16:00:00+00:00,79,100300,14.9,montaudran
95,12,2025-11-02T15:00:00+00:00,75,100200,15.7,montaudran


besoin métier :
    Utilisateur rentre une (ville,station) et cela affiche automatiquement les trois derniers jours, dès préfiltres peuvent etre appliqué sur la derniere semaine, last month, last years

    Donc class ville, station, record et voir si une class temps peut etre utile 

Ville 

In [ ]:
class Ville:
    def __init__(self, ville, df):
        self.ville = ville
        self.df = df       


    def filtrer_ville(self, colonne_nom_ville):
        """Retourne les lignes du DataFrame correspondant à la ville choisie"""
        return self.df[self.df[colonne_nom_ville].str.contains(self.ville, case=False, na=False)]

In [9]:
# station_filter.py
import pandas as pd
from extractors.base_extractor import BaseExtractor

class Station:
    def __init__(self, station: str, extractor: BaseExtractor):
        self.station = station
        self.extractor = extractor  # on dépend d’une abstraction, pas d’un CSV
        self.df = self.extractor.extract()

    def filtrer_station(self, colonne_nom_station="station_name") -> pd.DataFrame:
        """Retourne les lignes du DataFrame correspondant à la station"""
        if colonne_nom_station not in self.df.columns:
            print(f"⚠️ Colonne '{colonne_nom_station}' introuvable.")
            return pd.DataFrame()

        return self.df[self.df[colonne_nom_station].str.contains(self.station, case=False, na=False)]



In [24]:
from extractors.csv_extractor import CSVExtractor
from extractors.stations_config import STATIONS
# Crée l’extracteur (lecture CSV)
extractor = CSVExtractor("data_meteo_toulouse_station/data_all_stations.csv")

# Crée une instance de Station avec dépendance inversée
s = Station("montaudran", extractor)

# Filtrage
resultat = s.filtrer_station()
print(resultat.head())


✅ Fichier CSV chargé avec succès
   id             heure_de_paris  humidite  pression  temperature_en_degre_c  \
0  12  2025-11-06T14:00:00+00:00        92     98900                    12.9   
1  12  2025-11-06T13:00:00+00:00        92     98900                    12.7   
2  12  2025-11-06T12:00:00+00:00        93     98900                    12.4   
3  12  2025-11-06T11:00:00+00:00        92     98900                    12.7   
4  12  2025-11-06T10:00:00+00:00        92     99000                    12.9   

  station_name  
0   montaudran  
1   montaudran  
2   montaudran  
3   montaudran  
4   montaudran  


In [18]:
class Record:
    def __init__(self, station_obj):
        """Reçoit une instance de Station"""
        self.station_obj = station_obj
        self.df = station_obj.df  # récupère les données brutes

    def get_records(self, station_name):
        """Récupère tous les enregistrements liés à une station"""
        return self.station_obj.filtrer(station_name)

    def moyenne_temperature(self, station_name):
        """Exemple : calcule la température moyenne d'une station"""
        records = self.get_records(station_name)
        if "temperature_en_degre_c" in records.columns:
            return records["temperature_en_degre_c"].mean()
        else:
            print("⚠️ Colonne température manquante.")
            return None

In [23]:
r = Record(s)

# 3. Obtenir les données d’une station
records_montaudran = r.get_records("montaudran")
print(records_montaudran.head())

# 4. Calculer la moyenne
print("Température moyenne :", r.moyenne_temperature("montaudran"))


AttributeError: 'Station' object has no attribute 'filtrer'